<a href="https://colab.research.google.com/github/farhadhsn8/MLP/blob/master/implementationDynamicMLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Attribute Information:

1. sepal length in cm
2. sepal width in cm
3. petal length in cm
4. petal width in cm

## class:
1. Iris Setosa
2. Iris Versicolour
3. Iris Virginica

In [1]:
import matplotlib.pyplot as plt
from sklearn import datasets
import random
import numpy as np
import math
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [220]:
class MLP:

#   PARAMS =  {
#     # enter learning rate :0.1
#     # enter code of function for layer0 =>[ 1.sigmoid  | 2.tanh  | 3.relu | 4.linear ] :4
#     # enter number of Perceptrons for  layer 1 (start layer number from 0) : 2
#     # enter code of function for layer1 =>[ 1.sigmoid  | 2.tanh  | 3.relu | 4.linear ] :4
#     # enter code of function for layer2 =>[ 1.sigmoid  | 2.tanh  | 3.relu | 4.linear ] :4

#     'LEARNING_RATE' : 0.1 ,
#     'CODE_OF_ACTIVATION_FUNCTIONS' : [4,4,4] ,
#     'NUMBER_OF_PERCEPTRONS_FOR_HIDDEN_LAYERS' : [2]
#  }

  def __init__(self , feacturesOftrainingdata , lablesOftrainingdata , parameters):
    self.parameters = parameters
    self.feacturesOftrainingdata = feacturesOftrainingdata
    self.lablesOftrainingdata = lablesOftrainingdata
    self.slidingHead = 0
    self.etha = self.parameters['LEARNING_RATE']
    self.numberOfLayers = len(self.parameters['CODE_OF_ACTIVATION_FUNCTIONS'])
    self.layers =  np.empty(self.numberOfLayers,dtype=Layer)
    self.makeLayers()
    
    

  def makeLayers(self):
    for i in range(self.numberOfLayers):
      self.layers[i] = Layer(i , self)

  def train(self):
    return self.calculateMLP_outputForRow_k(self.getCurrentFeatureRow())


  def calculateMLP_outputForRow_k(self, X):
    self.resetAllcaches()
    return self.layers[self.numberOfLayers - 1].claculateLayerOutput(X)


  def getCurrentFeatureRow(self):
    return self.feacturesOftrainingdata[self.slidingHead]

  def getCurrentLableRow(self):
    return self.lablesOftrainingdata[self.slidingHead]


  def updateAllWeightsByBackPropagationAlgorithm(self):
    for layer in self.layers[:0:-1]:
      layer.updateLayerWeights( False)
    for layer in self.layers[:0:-1]:
      layer.updateLayerWeights( True)
    self.resetAllcaches()
    

  
  def trainingModel(self, epoch=1):
    
    for i in range(epoch):
      # printProgressBar(i, epoch, prefix = 'Progress:', suffix = 'Complete', length = 50)
      self.slidingHead =0
      for j in range(self.feacturesOftrainingdata.shape[0]):
        self.updateAllWeightsByBackPropagationAlgorithm()
        self.slidingHead +=1
      # printProgressBar(i + 1, epoch, prefix = 'Progress:', suffix = 'Complete', length = 50)

  def resetAllcaches(self):
    for i in range(self.numberOfLayers):
      self.layers[i].resetOutput()
      for j in range(self.layers[i].numberOfPerceptrons):
        self.layers[i].perceptrons[j].resetDelta()




  def clearAll(self):
      for i in range(self.numberOfLayers):
        self.layers[i].resetOutput()
        for j in range(self.layers[i].numberOfPerceptrons):
          self.layers[i].perceptrons[j].resetDelta()
          for k in range(self.layers[i].perceptrons[j].numberOfInputs):
            self.layers[i].perceptrons[j].inputBranchs[k].setW()
            print(self.layers[i].perceptrons[j].inputBranchs[k].w)
          

      

  



    



#--------------------------------------------------------------------------


class Layer:

  def __init__(self,layerAddress , MLP):
    self.MLP = MLP
    self.layerAddress = layerAddress
    self.numberOfPerceptrons = self.setNumberOfPerceptrons()
    self.activityFunction = ActivityFunction(self)
    self.perceptrons =  np.empty(self.numberOfPerceptrons,dtype=Perceptron)
    self.perceptrons = self.makePerceptrons()
    self.output = np.full((self.numberOfPerceptrons), math.inf)


  def resetOutput(self):
    self.output = np.full((self.numberOfPerceptrons), math.inf)


  def setNumberOfPerceptrons(self):
    if(self.layerAddress == 0 ):
      return  self.MLP.feacturesOftrainingdata.shape[1]
    if(self.layerAddress == self.MLP.numberOfLayers - 1 ):
      return  self.MLP.lablesOftrainingdata.shape[1]
    return self.MLP.parameters['NUMBER_OF_PERCEPTRONS_FOR_HIDDEN_LAYERS'][self.layerAddress-1]

  def makePerceptrons(self):
    perceptrons =  np.empty(self.numberOfPerceptrons,dtype=Perceptron) 
    for i in range( self.numberOfPerceptrons ):
      perceptrons[i] = Perceptron( i , self)
    return perceptrons

  def getPreviosLayer(self):
    return self.layerAddress != 0 and self.MLP.layers[self.layerAddress - 1 ] or -1

  
  def getNextLayer(self):
    return self.layerAddress != self.MLP.numberOfLayers - 1 \
     and self.MLP.layers[self.layerAddress + 1 ] or -1


  def claculateLayerOutput(self,X):     # receive Vector   # return Vector

    if ((any(self.output==math.inf))==False):
      return self.output
    if(self.layerAddress==0):
      X = X
      return X
    else:
      X = self.getPreviosLayer().claculateLayerOutput(X)
    output =  np.empty(self.numberOfPerceptrons)
    for i in range(self.numberOfPerceptrons):
      if(self.layerAddress == 0 ):
        output[i] = X[i]
      else:
        output[i] = self.perceptrons[i].calculatePerceptronOutput(X)
    self.output = output
    return self.output

  def calculateDerivativeOfActivationFunction(self,net):
    return self.activityFunction.calculateDerivative(net)

  def updateLayerWeights(self, hardUpdate = False):
    for perceptron in self.perceptrons:
      perceptron.updatePerceptronWeights(hardUpdate)
    

  



#--------------------------------------------------------------------------





class ActivityFunction:
  
  def __init__(self,layer):
    self.layer = layer
    self.functionType = self.layer.MLP.parameters['CODE_OF_ACTIVATION_FUNCTIONS'][self.layer.layerAddress]
  
  def runActivationFunction(self,x):
    if (self.functionType == 1) :
      return self.sigmoid(x)
    if (self.functionType == 2) :
      return self.tanh(x)
    if (self.functionType == 3) :
      return self.ReLU(x)
    if (self.functionType == 4) :
      return self.linear(x)

  def sigmoid(self, x):
    return 1 / (1 + math.exp(-x))

  def tanh(self , x):
    t=(math.exp(x)-math.exp(-x))/(math.exp(x)+math.exp(-x))
    return t

  def ReLU(self ,x):
    return max(0.0,x)

  def linear(self , x):
    return x

  def calculateDerivative(self , net):
    if (self.functionType == 1) :
      sig = self.sigmoid(net)
      return (1-sig)*sig
    if (self.functionType == 2) :
      return 1 - self.tanh(net)**2
    if (self.functionType == 3) :
      if(net<0):
        return 0
      return 1
    if (self.functionType == 4) :
      return 1



#--------------------------------------------------------------------------


class Perceptron:

  

  def __init__(self , perceptronNumber , layer ):   # [layerAddress  ,  perceptron] 
    self.baias = 0  # 0 or 1
    self.perceptronNumber = perceptronNumber
    self.layer = layer
    self.numberOfInputs  =  self.getNumberOfInputs()
    self.inputBranchs =  np.empty(self.numberOfInputs,dtype=Layer)
    self.makeInputs()
    self.delta = math.inf

  def resetDelta(self):
    self.delta = math.inf

  def makeInputs(self):
    for i in range(self.numberOfInputs):
      self.inputBranchs[i] = InputBranch(self , i)

  def getNumberOfInputs(self):
    if(self.layer.layerAddress == 0 ):
      return  1
    return self.layer.getPreviosLayer().numberOfPerceptrons + self.baias #@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@  return self.layer.getPreviosLayer().numberOfPerceptrons + 1 

  def calculatePerceptronOutput(self , X):
        net = self.calculatePerceptronNet(X)
        return self.layer.activityFunction.runActivationFunction(net)

        
  def calculatePerceptronNet(self , X):    # X is input feature vector
        y=0
        # DONT FORGET BAIAS
        X = np.concatenate((X, [self.baias]), axis=None)  #@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@   X = np.concatenate((X, [1]), axis=None) 
        for i in range(self.numberOfInputs):
          y += self.inputBranchs[i].calculateBranchOutput(X[i])
        return y


  def getDelta(self):
    if(self.delta != math.inf):
      return self.delta
    desiredOutput=0
    if(self.layer.layerAddress==self.layer.MLP.numberOfLayers - 1):
      desiredOutput = self.layer.MLP.getCurrentLableRow()[self.perceptronNumber]
    X = (self.layer.layerAddress == 0)  and self.layer.MLP.getCurrentFeatureRow() or self.layer.getPreviosLayer().claculateLayerOutput(self.layer.MLP.getCurrentFeatureRow())
    self.delta =  self.calculateDelta(X ,desiredOutput)
    return self.delta
    # print(self.layer.layerAddress, self.perceptronNumber,self.delta)

  def calculateDelta(self,X , desiredOutput):  # X is input vector 
    net = self.calculatePerceptronNet(X)
    if(self.layer.layerAddress == self.layer.MLP.numberOfLayers - 1):     # perceptron in output layer
      return self.layer.calculateDerivativeOfActivationFunction(net) * ( desiredOutput - self.calculatePerceptronOutput(X))
    else:       # perceptron in hidden layer
      sigma = 0
      # layerOutput = self.layer.claculateLayerOutput(self.layer.MLP.getCurrentFeatureRow())
      for perceptron in self.layer.getNextLayer().perceptrons:
        sigma += (perceptron.inputBranchs[self.perceptronNumber].w * perceptron.getDelta()) 
      return self.layer.calculateDerivativeOfActivationFunction(net) * sigma

  
  def updatePerceptronWeights(self,hardUpdate = False):
    for inputBranch in self.inputBranchs:
      hardUpdate and inputBranch.updateW() or inputBranch.updateWnew()


    


  
#--------------------------------------------------------------------------

class InputBranch:
  
  def __init__(self , perceptron, inputNumber):
    self.inputNumber = inputNumber
    self.perceptron = perceptron
    self.setW()
    self.Wnew = self.w

  def setW(self):
    if(self.perceptron.layer.layerAddress == 0):
      self.w =  1
    self.w = random.uniform(0,1) #00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

  def calculateBranchOutput(self , x):
    return self.w * x 

  

  def updateWnew(self):
    etha = self.perceptron.layer.MLP.etha
    yi = np.concatenate((self.perceptron.layer.getPreviosLayer().claculateLayerOutput(self.perceptron.layer.MLP.getCurrentFeatureRow()), [self.perceptron.baias]), axis=None)[self.inputNumber]  #@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    self.Wnew =self.w +  etha * self.perceptron.getDelta() * yi 

  def updateW(self):
    self.w = self.Wnew





In [671]:
iris = datasets.load_iris()
features = iris.data  
target = pd.get_dummies(iris.target).to_numpy()
features.shape   # (150, 4)
target.shape

dataset = np.hstack(( features,target,np.reshape(iris.target,(-1,1))))
#---------------shuffle---------------------
from sklearn.utils import shuffle
dataset=shuffle(dataset)

#-------------test & train ---------------
train=dataset[0:120,:]    
test=dataset[120:,:]  
test.shape                #(30, 7)
train.shape              # (120, 7)


# dataset[149]


(120, 8)

In [678]:
PARAMS = {
    # enter learning rate :0.1
    # enter code of function for layer0 =>[ 1.sigmoid  | 2.tanh  | 3.relu | 4.linear ] :4
    # enter number of Perceptrons for  layer 1 (start layer number from 0) : 2
    # enter code of function for layer1 =>[ 1.sigmoid  | 2.tanh  | 3.relu | 4.linear ] :4
    # enter code of function for layer2 =>[ 1.sigmoid  | 2.tanh  | 3.relu | 4.linear ] :4
    'LEARNING_RATE' : 0.01 ,
    'CODE_OF_ACTIVATION_FUNCTIONS' : [4,2,3] , #[ 1.sigmoid  | 2.tanh  | 3.relu | 4.linear ]
    'NUMBER_OF_PERCEPTRONS_FOR_HIDDEN_LAYERS' : [5]
  }

PARAMS['NUMBER_OF_PERCEPTRONS_FOR_HIDDEN_LAYERS']



[5]

In [679]:
IRIS_MLP = MLP(train[:,0:4] ,train[:,4:7] ,PARAMS )
# IRIS_MLP = MLP(train[:,0:4] ,5 * np.reshape(train[:,7],(-1,1,1)) ,PARAMS)
# IRIS_MLP = MLP(np.array([[1,1]]) ,np.array([[2,2]]) , PARAMS )
# IRIS_MLP = MLP(s1 ,s2, PARAMS )

In [680]:
# IRIS_MLP.clearAll()

In [681]:

IRIS_MLP.trainingModel(100)

In [683]:
s=0
k=0
for i in test: #test or train
  est = IRIS_MLP.calculateMLP_outputForRow_k(i[0:4])
  print(est , i[4:7])
  k+=int(np.argmax(est) == np.argmax(i[4:7]) )
  s+=1

  

k, s , str(k/s * 100)+'%'

 # # print([IRIS_MLP.calculateMLP_outputForRow_k(s1[i])[0] ,IRIS_MLP.calculateMLP_outputForRow_k(s1[i])[1]] , list(s2[i]))
 # # s+=int([int(IRIS_MLP.calculateMLP_outputForRow_k(s1[i])[0]>0) ,int(IRIS_MLP.calculateMLP_outputForRow_k(s1[i])[1]>0)] == list(s2[i]))
# print(IRIS_MLP.calculateMLP_outputForRow_k([1,1]) )

[0.99848164 0.17198087 0.        ] [1. 0. 0.]
[0.99823584 0.17201416 0.        ] [1. 0. 0.]
[0.         0.38055866 0.99072584] [0. 0. 1.]
[0.99604302 0.17218898 0.        ] [1. 0. 0.]
[0.99728197 0.17216161 0.        ] [1. 0. 0.]
[0.        0.3026091 0.       ] [0. 1. 0.]
[0.         0.33196034 0.17810801] [0. 1. 0.]
[0.         0.38240142 1.02152945] [0. 0. 1.]
[0.         0.33600394 0.24572055] [0. 1. 0.]
[0.98805872 0.17302123 0.        ] [1. 0. 0.]
[0.99898129 0.17227874 0.        ] [1. 0. 0.]
[0.         0.37917907 0.96763975] [0. 0. 1.]
[0.97976824 0.17394422 0.        ] [1. 0. 0.]
[0.         0.31213527 0.        ] [0. 1. 0.]
[0.99441637 0.17238797 0.        ] [1. 0. 0.]
[0.         0.36742499 0.77111898] [0. 0. 1.]
[0.         0.33139169 0.16921235] [0. 1. 0.]
[0.         0.38264939 1.0256649 ] [0. 0. 1.]
[0.         0.3705859  0.82401075] [0. 1. 0.]
[0.99665289 0.17220335 0.        ] [1. 0. 0.]
[0.         0.38167945 1.00944926] [0. 0. 1.]
[0.         0.38050263 0.98977212] [0

(29, 30, '96.66666666666667%')

In [684]:
for i in range(IRIS_MLP.numberOfLayers):
  print('layer'+str(i))
  for j in range(IRIS_MLP.layers[i].numberOfPerceptrons):
    print('percepron'+str(j))
    for k in range(IRIS_MLP.layers[i].perceptrons[j].numberOfInputs):
      print(IRIS_MLP.layers[i].perceptrons[j].inputBranchs[k].w)

layer0
percepron0
0.6745741915769821
percepron1
0.7228124777241237
percepron2
0.2240955526384436
percepron3
0.7604801556986462
layer1
percepron0
-0.2675212637673663
-0.7665263801452425
0.6370509415846145
1.1836632022822173
percepron1
0.4836541800763478
0.6836016974115433
0.1282189587269721
0.7420572263552115
percepron2
0.4805091716517456
0.7603537062079601
0.7408765129602126
0.6396585767737258
percepron3
0.3517275271994939
0.6309296244390415
0.7935077330435562
0.7092022370811348
percepron4
0.6145790530678699
0.060287918595537365
0.6798215817599988
0.46831036896129835
layer2
percepron0
-1.1161823602646326
-0.05089353945017402
-0.08852231696908709
0.387556356525203
-0.3612706469169865
percepron1
0.10699432331504279
0.45662882565550805
0.3094868202759435
-0.040245292057601134
-0.4474039319375628
percepron2
1.7890402397912282
0.3079120254786013
-0.5599813758330291
0.0028568307193348606
-0.4671549026249559
